In [1]:
import os
import json
import re
import pandas as pd
import numpy as np

In [2]:
data_path = 'result/senator_answer_original_gpt3.5_0613.json'
with open(data_path, 'r') as f:
    data = f.readlines()
print(len(data))

500


In [11]:
# for every line in data, extract the answer
answer_list = []
for line in data:
    line = json.loads(line)
    answer_list.append(line['answer'])
print(len(answer_list))

500


In [39]:
question_list = []
for line in data:
    line = json.loads(line)
    question_list.append(line['question'])
print(len(question_list))

500


In [65]:
wrong_answer_count = 0
senator_name_dict = {}
bad_words = ['United States', 'University','College','School','Senator','Institute','Virginia','Biology','Industry','Security','Science'
'Massachusetts','California','Georgia','Eastern','Although','Chicago','Washington','New York','Texas','Florida','Pennsylvania','Business','Dakota', 'Santa Barbara',
'Science','Technology','Engineering','Mathematics','Computer','Information','Systems','Management','Administration','Accounting','Finance','Physics',
'New Jersey','Carolina','Ohio','Michigan','Illinois','Indiana','Wisconsin','Minnesota','Missouri','Colorado','Arizona','Tennessee',
'Maryland','Massachusetts','Alabama','Louisiana','Kentucky','Oregon','Oklahoma','Connecticut','Iowa','Utah','Nevada','Arkansas','Mississippi',
'South Carolina','Kansas','New Mexico','Nebraska','West Virginia','Idaho','Hawaii','New Hampshire','Maine','Rhode Island','Montana','Delaware','Los Angeles','Detroit',
'Doctor','Law','Johns Hopkins','Harvard','Yale','Princeton','Stanford','Columbia','Duke','Cornell','Berkeley','MIT','Chicago','Northwestern',
'Republican','Public','Private','Democrat','Libertarian','Independent','Party','Senate','House','Representatives','Congress','Committee','Political','Scholar','Medical Center'
'History','City','American','International']
for answer in answer_list:
    index = answer_list.index(answer)
    if 'Yes' in answer:
        # print(answer)
        # found senator's name
        wrong_answer_count += 1
        senator_name = re.findall(r'([A-Z][a-z]+ [A-Z][a-z]+)', answer)
        # print(senator_name)
        # delete school name and University name and United States
        for i in range(len(senator_name)-1,-1,-1):
            # print(senator_name[i],end='|| ')
            for bad_word in bad_words:
                if bad_word in senator_name[i]:
                    senator_name.pop(i)
                    break
        # print(senator_name)
        senator_name = list(set(senator_name))
        senator_name_dict[index] = senator_name



In [54]:
state_university_list = []
for question in question_list:
    # print(question)
    # state of 后面的词 和 alma mater 后面的词
    state = re.findall(r'state of ([A-Z][a-z]+)', question)
    alma_mater = re.findall(r'mater was ([A-Za-z ]+)', question)

    list1 = [state, alma_mater]
    state_university_list.append(list1)


In [59]:
print(len(state_university_list))
print(len(senator_name_dict))

500
287


In [67]:
import openai
import time


In [75]:
openai_key = os.getenv('OPENAI_API_KEY')
openai.api_key = openai_key

In [80]:
recognized = 0
for i in senator_name_dict.keys():
    for senator in senator_name_dict[i]:
        verify_question_state = 'Has ' + senator + ' represented the state of ' + state_university_list[i][0][0] + ' as US senator?'
        verify_question_university = 'Was ' + senator + '\'s alma mater ' + state_university_list[i][1][0] + '?' 
        print(verify_question_university)
        try:
            completion = openai.ChatCompletion.create(
                model = 'gpt-3.5-turbo-0613',
                messages = [{'role':'user','content':verify_question_state}],
            )
        except:
            time.sleep(60)
            completion = openai.ChatCompletion.create(
                model = 'gpt-3.5-turbo-0613',
                messages = [{'role':'user','content':verify_question_state}],
            )
        answer = completion.choices[0].message['content']
        print(answer)
        with open('./result/senator_answer_verification_state_gpt3.5_0613.json', 'a+') as f:
            answer_dict = {}
            answer_dict['question'] = verify_question_state
            answer_dict['answer'] = answer
            f.write(json.dumps(answer_dict) + '\n')
        if 'No' in answer:
            recognized += 1
            break
        try:
            completion = openai.ChatCompletion.create(
                model = 'gpt-3.5-turbo-0613',
                messages = [{'role':'user','content':verify_question_university}],
            )
        except:
            # print what is wrong
            time.sleep(60)
            completion = openai.ChatCompletion.create(
                model = 'gpt-3.5-turbo-0613',
                messages = [{'role':'user','content':verify_question_university}],
            )
        answer = completion.choices[0].message['content']
        print(answer)
        with open('./result/senator_answer_verification_university_gpt3.5_0613.json', 'a+') as f:
            answer_dict = {}
            answer_dict['question'] = verify_question_university
            answer_dict['answer'] = answer
            f.write(json.dumps(answer_dict) + '\n')
        if 'No' in answer:
            recognized += 1
            break
print(recognized)
        


Was Richard Shelby's alma mater MIT?
Yes, Richard Shelby has represented the state of Alabama as a US senator. He has served as a US senator from Alabama since 1987 and continues to serve in that capacity.
No, Richard Shelby's alma mater is the University of Alabama, where he earned his bachelor's and law degrees.
Was Ted Stevens's alma mater MIT?
Yes, Ted Stevens represented the state of Alaska as a United States Senator. He served as a Senator from 1968 to 2009.
No, Ted Stevens's alma mater was not MIT (Massachusetts Institute of Technology). He graduated from the University of California, Los Angeles (UCLA) with a bachelor's degree in political science.
Was Mark Pryor's alma mater MIT?
Yes, Mark Pryor has represented the state of Arkansas as a US senator. He served as a US senator from Arkansas from 2003 to 2015, representing the Democratic Party.
No, Mark Pryor's alma mater is the University of Arkansas.
Was Timothy Wirth's alma mater MIT?
Yes, Timothy Wirth represented the state o

In [85]:
print(recognized)
print(1 - wrong_answer_count/500)

225
0.42600000000000005
